In [5]:
# $ pip install requests
import requests

In [6]:
# $ pip install bs4
from bs4 import BeautifulSoup

In [7]:
# pip install python-forecastio
import forecastio

- api json을 사용: 네이버 주식 데이터 크롤링 할 것임.
- api json을 사용, forecastio: dark sky api로 날씨 데이터를 수집하겠습니다.
- bs4 : 네이버 실시간 키워드, 다음 실시간 키워드 크롤링
- web file url로 파일을 다운로드 받는 방법

## 1. 네이버 주식 데이터 크롤링

In [8]:
### 보통 모바일 페이지가 더 단순함
## http://m.stock.naver.com/sise/siseList.nhn?menu=market_sum&sosok=0
## Network -> XHR
## Header:
## Remote Address: 실제 찾아간 IP
## Request header: 우리가 반대편에 Reqeust할 때, 보내는 정보 
## 여기서 중요한 것은 Cookie:인데, 
## Referer: 이전 페이지 
## Request Header를 수정 할 수 있다 -> 중요할 듯함. 
## Response는 문자열로 받음
## http://m.stock.naver.com/api/json/sise/siseListJson.nhn?menu=market_sum&sosok=0&pageSize=20&page=1
## http://m.stock.naver.com/api/json/sise/siseListJson.nhn?menu=market_sum&sosok=0&pageSize=20&page=1
## http://m.stock.naver.com/api/json/sise/siseListJson.nhn?menu=market_sum&sosok=0&pageSize=1458&page=1 한페이지에 1458개 종목 다보기
## 1은 코스피, 2는 코스닥 

In [13]:
def make_url(pageSize=10, page=1):
    return 'http://m.stock.naver.com/api/json/sise/siseListJson.nhn?menu=market_sum&sosok=0&pageSize=' + str(pageSize) + '&page=' + str(page) ## 1458&page=1'

make_url()

'http://m.stock.naver.com/api/json/sise/siseListJson.nhn?menu=market_sum&sosok=0&pageSize=10&page=1'

In [70]:
from pandas.io.json import json_normalize # for문 안써도 되게 하는 패키지

In [71]:
# 데이터를 가져와서 판다스의 데이타 프레임으로 만드는 함수
def get_data(url):
    response = requests.get(url)
    json_info = response.json() # string 데이터를, json 으로 parsing 함
    companys = json_info['result']['itemList']
    return json_normalize(companys)
#     df = pd.DataFrame(columns = ['종목', '시세', '전일비', '등락율', '시가총액', '거래량'])
    
#     for company in companys:
#         df.loc[len(df)] = {
#             '종목': company['nm'],
#             '시세': company['nv'],
#             '전일비': company['cv'],
#             '등락율': company['cr'],
#             '시가총액': company['mks'],
#             '거래량': company['aq'],
#         }
        
#     return df

In [73]:
url = make_url(1500, 1)
df = get_data(url)
df.tail()

,aa,aq,cd,cr,cv,mks,mt,nm,nv,pcv,rf
1453,79,8033,284990,-0.10,-10,22,0,KBSTAR 200에너지화학,9875,9885,5
1454,2,225,284980,-1.78,-175,21,0,KBSTAR 200금융,9680,9855,5
1455,0,50,295820,-0.80,-80,15,0,ARIRANG 200동일가중,9940,10020,5
1456,280,28489,295860,-1.11,-110,15,0,ARIRANG 200모멘텀,9770,9880,5
1457,278,28667,295890,-1.23,-120,14,0,ARIRANG 200로우볼,9665,9785,5


In [29]:
# 데이터를 가져와서 판다스의 데이타 프레임으로 만드는 함수
def get_data(url):
    response = requests.get(url)
    json_info = response.json() # string 데이터를, json 으로 parsing 함
    companys = json_info['result']['itemList']
    
    df = pd.DataFrame(columns = ['종목', '시세', '전일비', '등락율', '시가총액', '거래량'])
    
    for company in companys:
        df.loc[len(df)] = {
            '종목': company['nm'],
            '시세': company['nv'],
            '전일비': company['cv'],
            '등락율': company['cr'],
            '시가총액': company['mks'],
            '거래량': company['aq'],
        }
        
    return df

In [32]:
url = make_url(1500, 1)
df = get_data(url)
df.tail()

,종목,시세,전일비,등락율,시가총액,거래량
1453,KBSTAR 200에너지화학,9875,-10,-0.10,22,8033
1454,KBSTAR 200금융,9680,-175,-1.78,21,225
1455,ARIRANG 200동일가중,9940,-80,-0.80,15,50
1456,ARIRANG 200모멘텀,9770,-110,-1.11,15,28489
1457,ARIRANG 200로우볼,9665,-120,-1.23,14,28667


#### 공식적으로 제공해주는 API 데이터를 가져와 보겠습니다.
##### Dark sky api
- 위도와 경도를 입력하면 날씨 정보를 보내주는 api
- https://darksky.net/dev

In [33]:
FORECAST_TOKEN = '12b8cd89392777d5f1a9b56e91d172d5'

In [48]:
def forecast(lat, lng): 
    url = 'https://api.darksky.net/forecast/{}/{},{}'.format(FORECAST_TOKEN, lat, lng)
    response = requests.get(url)
    json_info = response.json()
    return json_info['timezone'], json_info['hourly']['summary']

In [49]:
lat = 37.512
lng = 126.954 
forecast(lat, lng)

('Asia/Seoul', 'Mostly cloudy throughout the day.')

In [50]:
def forecast2(lat, lng):
    forecast = forecastio.load_forecast(FORECAST_TOKEN, lat, lng)
    byHourly = forecast.hourly()
    return byHourly.summary

In [51]:
lat = 37.512
lng = 126.954
forecast2(lat, lng)

'Mostly cloudy throughout the day.'

#### bs4
- 네이버 키워드 랭킹 데이터 크롤링
- 다음 키워드 랭킹 데이터 크롤링

In [104]:
##  언제 쓸 수 있을까? 네이버와 다음 둘다 랭킹의 올라가는 속도가 빠르면
## hot issue이고 시장의 영향을 이미 주었거나, 주는중인 issue로 볼 수 있을듯.

# dom.select - 여러개의 html 엘리먼트를 셀렉팅 할 때 사용, 결과로 리스트 데이터를 리턴.
# dom.select_one - 하나의 html 엘리먼트를 셀렉팅 할 때 사용, 결과로 문자열 데이터를 리턴.

def naver():
    df = pd.DataFrame(columns = ['rank', 'keyword'])
    
    response = requests.get('https://www.naver.com/') ## html 코드 가져옴. # BS는 response.content이용
    dom = BeautifulSoup(response.content, 'html.parser') # html 형태로 파싱 후, dom객체에 저장
    
    keywords = dom.select('.ah_roll_area > .ah_l > .ah_item') # css select를 어떻게 확인하냐면, 
#     print(len(keywords)) 
    
    for keyword in keywords:
        df.loc[len(df)] = {
            'rank': keyword.select_one('.ah_r').text,
            'keyword': keyword.select_one('.ah_k').text,
        }
    
    return df
naver_df = naver()
naver_df['keyword']

0          이집트 우루과이
1             포항 약국
2              뮤직뱅크
3              블랙핑크
4              광주날씨
5     러시아 사우디 하이라이트
6               강혁민
7            모로코 이란
8               최원명
9                살라
10              박기량
11               쌈디
12              민갑룡
13              이찬오
14              심영은
15         포르투갈 스페인
16              배용제
17           프로듀스48
18              ksl
19            국가장학금
Name: keyword, dtype: object

In [65]:
### 한시간에 한번 씩 돌리고 싶을 때
import time

ls = []
for idx in range(3):
    print(idx)
    ls.append(naver())
    time.sleep(5) ### 5초에 한번 씩


0
1
2


#### Daum 실시간 검색어

In [105]:
def daum():
    df = pd.DataFrame()
    rank = []
    keywords = []
    
    response = requests.get('https://www.daum.net/')
    dom = BeautifulSoup(response.content, 'html.parser')
    keywords = dom.select('.realtime_part > .list_hotissue.issue_row > li')
    rank = [keyword.select_one('.ir_wa').text[:-1] for keyword in keywords ]
    keyword = [keyword.select_one('.link_issue').text for keyword in keywords ]
    df['rank'] = rank
    df['keyword'] = keyword
    return df

daum_df = daum()
daum_df['keyword']

0       민갑룡
1      경찰청장
2     국가장학금
3    한국장학재단
4       조현배
5      블랙핑크
6       안희정
7       김무성
8       심영은
9       이찬오
Name: keyword, dtype: object

In [106]:
# naver_df['keyword'] , daum_df['keyword']

result = [
    keyword
    for keyword in naver_df['keyword']
    if daum_df['keyword'].str.contains(keyword).any()
]

result

['블랙핑크', '민갑룡', '이찬오', '심영은', '국가장학금']

In [112]:
set(naver_df['keyword']) & set(daum_df['keyword'])

{'국가장학금', '민갑룡', '블랙핑크', '심영은', '이찬오'}

#### File down load : requests
- file의 url을 받아서 해당 url의 file을 다운로드 하는 방법

In [119]:
def download(title, download_link):
    response = requests.get(download_link, stream=True) # stream을 True로 해야 함. -> 파일을 다운 받는 것이기에
    file_size = 0
    with open(title, 'wb') as f: # Binary로 함
        for chunk in response.iter_content(chunk_size=1024): # byte
            if chunk:
                file_size += 1024
                f.write(chunk)
        return file_size

In [121]:
# url = 'https://r3---sn-pj2-2v1e.googlevideo.com/videoplayback?sparams=dur%2Cei%2Cid%2Cinitcwndbps%2Cip%2Cipbits%2Citag%2Clmt%2Cmime%2Cmm%2Cmn%2Cms%2Cmv%2Cpcm2cms%2Cpl%2Cratebypass%2Crequiressl%2Csource%2Cexpire&ipbits=0&signature=E206469B2D89166376D7525E05196CF56FF266D4.8BE8C02D86DBF48F5CCEE7008BC2582977DD194C&pl=17&itag=22&mime=video%2Fmp4&c=WEB&pcm2cms=yes&mn=sn-pj2-2v1e%2Csn-f5f7lnee&mm=31%2C29&ratebypass=yes&requiressl=yes&mv=m&mt=1529048870&ms=au%2Crdu&ei=qm8jW_THC5rK7QTkgISYDA&lmt=1529048830232204&key=yt6&ip=77.65.13.26&expire=1529070602&dur=208.399&id=o-AD0RhPvbWjOTimLTuGqwfMKjXnFIZW1BIL7ukuLEHDlr&initcwndbps=1205000&source=youtube&fvip=3&video_id=ZUWGAvcqOdg&title=%5B%ED%95%98%ED%8A%B8%EC%8B%9C%EA%B7%B8%EB%84%902+%EC%84%A0%EA%B3%B5%EA%B0%9C%5D+%EC%8F%98-%EC%8A%A4%EC%9C%97%ED%95%9C+%EB%8F%84%EA%B7%A0+%27%EB%81%9D%EC%9D%B4%EB%9D%BC%EA%B3%A0%EB%8A%94+%EC%83%9D%EA%B0%81+%EC%95%88+%ED%95%B4%27+-+%EC%B1%84%EB%84%90A+%ED%95%98%ED%8A%B8%EC%8B%9C%EA%B7%B8%EB%84%902+13%ED%9A%8C'
download_link = 'https://bit.ly/2taHh9E'
title = 'HeartSignal.mp4'
file_size = download(title, download_link)

In [122]:
file_size

14526464

In [126]:
str(round(file_size/1024/1024,2)) + "Mb"

'13.85Mb'

- bit => 0 또는 1
- byte => 8 bits
- 1kbyte => 1024 bytes
- 1Mbyte => 1024 kbytes